In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 70*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240410163047
0	Validation losses: 0.4750, 35.0067, 35.4817	Best Loss: 35.4817 (0)	Accuracy: 82.00%
1	Validation losses: 0.4386, 9.5979, 10.0365	Best Loss: 10.0365 (0)	Accuracy: 85.20%
2	Validation losses: 0.4798, 1.2969, 1.7767	Best Loss: 1.7767 (0)	Accuracy: 83.14%
3	Validation losses: 0.5111, 0.4084, 0.9196	Best Loss: 0.9196 (0)	Accuracy: 81.94%
4	Validation losses: 0.4901, 0.3447, 0.8349	Best Loss: 0.8349 (0)	Accuracy: 82.62%
5	Validation losses: 0.5214, 0.3190, 0.8404	Best Loss: 0.8349 (1)	Accuracy: 80.40%
6	Validation losses: 0.4786, 0.2734, 0.7521	Best Loss: 0.7521 (0)	Accuracy: 83.54%
7	Validation losses: 0.6020, 0.2479, 0.8499	Best Loss: 0.7521 (1)	Accuracy: 78.72%
8	Validation losses: 0.5391, 0.2051, 0.7442	Best Loss: 0.7442 (0)	Accuracy: 81.06%
9	Validation losses: 0.5380, 0.2008, 0.7388	Best Loss: 0.7388 (0)	Accuracy: 81.94%
10	Validation losses: 0.6602, 0.1726, 0.8328	Best Loss: 0.7388 (1)	Accuracy: 78.38%
11	Validation losses: 0.

23	Validation losses: 2.3061, 0.3614, 2.6675	Best Loss: 0.6710 (6)	Accuracy: 9.86%
24	Validation losses: 2.3045, 0.3500, 2.6545	Best Loss: 0.6710 (7)	Accuracy: 9.86%
25	Validation losses: 2.3053, 0.3427, 2.6480	Best Loss: 0.6710 (8)	Accuracy: 9.86%
26	Validation losses: 2.3057, 0.3386, 2.6442	Best Loss: 0.6710 (9)	Accuracy: 9.14%
27	Validation losses: 2.3027, 0.3344, 2.6371	Best Loss: 0.6710 (10)	Accuracy: 9.76%
28	Validation losses: 2.3044, 0.3310, 2.6354	Best Loss: 0.6710 (11)	Accuracy: 9.80%
29	Validation losses: 2.3041, 0.3288, 2.6329	Best Loss: 0.6710 (12)	Accuracy: 10.08%
30	Validation losses: 2.3037, 0.3270, 2.6307	Best Loss: 0.6710 (13)	Accuracy: 10.08%
31	Validation losses: 2.3028, 0.3252, 2.6280	Best Loss: 0.6710 (14)	Accuracy: 11.12%
32	Validation losses: 2.3039, 0.3239, 2.6279	Best Loss: 0.6710 (15)	Accuracy: 9.14%
33	Validation losses: 2.3034, 0.3224, 2.6258	Best Loss: 0.6710 (16)	Accuracy: 10.02%
34	Validation losses: 2.3039, 0.3211, 2.6250	Best Loss: 0.6710 (17)	Accuracy

1	Validation losses: 0.4084, 12.4561, 12.8644	Best Loss: 12.8644 (0)	Accuracy: 85.60%
2	Validation losses: 0.4170, 3.3784, 3.7953	Best Loss: 3.7953 (0)	Accuracy: 85.26%
3	Validation losses: 0.4800, 0.7204, 1.2004	Best Loss: 1.2004 (0)	Accuracy: 82.12%
4	Validation losses: 0.4780, 0.3242, 0.8022	Best Loss: 0.8022 (0)	Accuracy: 82.42%
5	Validation losses: 0.5015, 0.2822, 0.7837	Best Loss: 0.7837 (0)	Accuracy: 81.30%
6	Validation losses: 0.4750, 0.2598, 0.7349	Best Loss: 0.7349 (0)	Accuracy: 82.78%
7	Validation losses: 0.4944, 0.2440, 0.7384	Best Loss: 0.7349 (1)	Accuracy: 81.90%
8	Validation losses: 0.4882, 0.2037, 0.6919	Best Loss: 0.6919 (0)	Accuracy: 82.76%
9	Validation losses: 0.8170, 0.1721, 0.9891	Best Loss: 0.6919 (1)	Accuracy: 71.58%
10	Validation losses: 0.4758, 0.1530, 0.6288	Best Loss: 0.6288 (0)	Accuracy: 83.48%
11	Validation losses: 0.4643, 0.1323, 0.5966	Best Loss: 0.5966 (0)	Accuracy: 85.30%
12	Validation losses: 0.4709, 0.1261, 0.5970	Best Loss: 0.5966 (1)	Accuracy: 83.34

60	Validation losses: 0.4083, 0.0983, 0.5067	Best Loss: 0.4538 (19)	Accuracy: 85.56%
61	Validation losses: 0.4162, 0.0964, 0.5126	Best Loss: 0.4538 (20)	Accuracy: 85.78%
Early stopping!
Total running time:	 4939
INFO:tensorflow:Restoring parameters from ./models/model_20240410211346.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    13.9699995518%
RANDOM SEED:   2 	LAMBDA_EXP:   -4.4
Now: 20240410223607
0	Validation losses: 0.4102, 19.0808, 19.4910	Best Loss: 19.4910 (0)	Accuracy: 85.48%
1	Validation losses: 0.3794, 12.0545, 12.4339	Best Loss: 12.4339 (0)	Accuracy: 86.42%
2	Validation losses: 0.4080, 5.5200, 5.9280	Best Loss: 5.9280 (0)	Accuracy: 85.72%
3	Validation losses: 0.5394, 1.8317, 2.3711	Best Loss: 2.3711 (0)	Accuracy: 80.58%
4	Validation losses: 0.4000, 0.5691, 0.9692	Best Loss: 0.9692 (0)	Accuracy: 85.58%
5	Validation losses: 0.4603, 0.2858, 0.7461	Best Loss: 0.7461 (0)	Accurac

14	Validation losses: 0.5480, 0.1298, 0.6778	Best Loss: 0.5585 (1)	Accuracy: 80.46%
15	Validation losses: 0.4815, 0.1506, 0.6321	Best Loss: 0.5585 (2)	Accuracy: 84.26%
16	Validation losses: 0.4678, 0.1201, 0.5879	Best Loss: 0.5585 (3)	Accuracy: 84.48%
17	Validation losses: 0.4639, 0.1122, 0.5761	Best Loss: 0.5585 (4)	Accuracy: 83.14%
18	Validation losses: 0.4363, 0.1062, 0.5426	Best Loss: 0.5426 (0)	Accuracy: 84.74%
19	Validation losses: 0.4210, 0.1031, 0.5241	Best Loss: 0.5241 (0)	Accuracy: 86.44%
20	Validation losses: 0.6165, 0.1142, 0.7307	Best Loss: 0.5241 (1)	Accuracy: 81.66%
21	Validation losses: 0.4576, 0.1068, 0.5644	Best Loss: 0.5241 (2)	Accuracy: 84.76%
22	Validation losses: 0.4034, 0.1024, 0.5058	Best Loss: 0.5058 (0)	Accuracy: 84.86%
23	Validation losses: 0.3979, 0.0994, 0.4973	Best Loss: 0.4973 (0)	Accuracy: 86.46%
24	Validation losses: 0.4099, 0.0976, 0.5075	Best Loss: 0.4973 (1)	Accuracy: 85.68%
25	Validation losses: 0.4069, 0.0960, 0.5029	Best Loss: 0.4973 (2)	Accuracy:

...

23	Validation losses: 0.3076, 0.2155, 0.5231	Best Loss: 0.5231 (0)	Accuracy: 89.62%
24	Validation losses: 0.3213, 0.2089, 0.5302	Best Loss: 0.5231 (1)	Accuracy: 89.48%
25	Validation losses: 0.3371, 0.2028, 0.5400	Best Loss: 0.5231 (2)	Accuracy: 89.26%
26	Validation losses: 0.3220, 0.1972, 0.5191	Best Loss: 0.5191 (0)	Accuracy: 90.04%
27	Validation losses: 0.3524, 0.1919, 0.5443	Best Loss: 0.5191 (1)	Accuracy: 89.76%
28	Validation losses: 0.3443, 0.1872, 0.5315	Best Loss: 0.5191 (2)	Accuracy: 89.94%
29	Validation losses: 0.3363, 0.1829, 0.5192	Best Loss: 0.5191 (3)	Accuracy: 90.18%
30	Validation losses: 0.3747, 0.1788, 0.5535	Best Loss: 0.5191 (4)	Accuracy: 89.64%
31	Validation losses: 0.3656, 0.1750, 0.5407	Best Loss: 0.5191 (5)	Accuracy: 90.32%
32	Validation losses: 0.3902, 0.1716, 0.5618	Best Loss: 0.5191 (6)	Accuracy: 90.20%
33	Validation losses: 0.3965, 0.1685, 0.5650	Best Loss: 0.5191 (7)	Accuracy: 90.86%
34	Validation losses: 0.4100, 0.1658, 0.5758	Best Loss: 0.5191 (8)	Accuracy:

22	Validation losses: 0.3186, 0.1874, 0.5061	Best Loss: 0.5029 (6)	Accuracy: 89.20%
23	Validation losses: 0.3235, 0.1847, 0.5082	Best Loss: 0.5029 (7)	Accuracy: 89.90%
24	Validation losses: 0.3316, 0.1820, 0.5136	Best Loss: 0.5029 (8)	Accuracy: 89.78%
25	Validation losses: 0.3407, 0.1790, 0.5197	Best Loss: 0.5029 (9)	Accuracy: 89.78%
26	Validation losses: 0.3595, 0.1758, 0.5353	Best Loss: 0.5029 (10)	Accuracy: 89.76%
27	Validation losses: 0.3331, 0.1732, 0.5063	Best Loss: 0.5029 (11)	Accuracy: 89.90%
28	Validation losses: 0.3798, 0.1707, 0.5505	Best Loss: 0.5029 (12)	Accuracy: 89.88%
29	Validation losses: 0.3402, 0.1682, 0.5085	Best Loss: 0.5029 (13)	Accuracy: 90.60%
30	Validation losses: 0.3843, 0.1660, 0.5503	Best Loss: 0.5029 (14)	Accuracy: 90.52%
31	Validation losses: 0.4260, 0.1638, 0.5898	Best Loss: 0.5029 (15)	Accuracy: 90.16%
32	Validation losses: 0.3911, 0.1618, 0.5530	Best Loss: 0.5029 (16)	Accuracy: 90.56%
33	Validation losses: 0.4293, 0.1601, 0.5894	Best Loss: 0.5029 (17)	A

8	Validation losses: 0.3118, 0.1451, 0.4570	Best Loss: 0.4570 (0)	Accuracy: 88.80%
9	Validation losses: 0.3298, 0.1449, 0.4747	Best Loss: 0.4570 (1)	Accuracy: 88.60%
10	Validation losses: 0.3215, 0.1448, 0.4663	Best Loss: 0.4570 (2)	Accuracy: 88.10%
11	Validation losses: 0.3542, 0.1446, 0.4989	Best Loss: 0.4570 (3)	Accuracy: 87.66%
12	Validation losses: 0.3322, 0.1444, 0.4766	Best Loss: 0.4570 (4)	Accuracy: 88.46%
13	Validation losses: 0.3143, 0.1442, 0.4585	Best Loss: 0.4570 (5)	Accuracy: 88.94%
14	Validation losses: 0.3277, 0.1439, 0.4716	Best Loss: 0.4570 (6)	Accuracy: 88.50%
15	Validation losses: 0.3249, 0.1439, 0.4688	Best Loss: 0.4570 (7)	Accuracy: 88.74%
16	Validation losses: 0.3100, 0.1438, 0.4538	Best Loss: 0.4538 (0)	Accuracy: 89.16%
17	Validation losses: 0.3172, 0.1437, 0.4608	Best Loss: 0.4538 (1)	Accuracy: 89.20%
18	Validation losses: 0.3385, 0.1436, 0.4821	Best Loss: 0.4538 (2)	Accuracy: 89.26%
19	Validation losses: 0.3298, 0.1437, 0.4735	Best Loss: 0.4538 (3)	Accuracy: 8

1	Validation losses: 0.3600, 0.0923, 0.4522	Best Loss: 0.4522 (0)	Accuracy: 86.88%
2	Validation losses: 0.3560, 0.0923, 0.4484	Best Loss: 0.4484 (0)	Accuracy: 87.22%
3	Validation losses: 0.3359, 0.0925, 0.4284	Best Loss: 0.4284 (0)	Accuracy: 87.04%
4	Validation losses: 0.3269, 0.0926, 0.4196	Best Loss: 0.4196 (0)	Accuracy: 88.44%
5	Validation losses: 0.3283, 0.0928, 0.4212	Best Loss: 0.4196 (1)	Accuracy: 88.18%
6	Validation losses: 0.2972, 0.0931, 0.3903	Best Loss: 0.3903 (0)	Accuracy: 89.04%
7	Validation losses: 0.3205, 0.0933, 0.4138	Best Loss: 0.3903 (1)	Accuracy: 88.70%
8	Validation losses: 0.3166, 0.0936, 0.4103	Best Loss: 0.3903 (2)	Accuracy: 88.64%
9	Validation losses: 0.3709, 0.0940, 0.4649	Best Loss: 0.3903 (3)	Accuracy: 87.46%
10	Validation losses: 0.3512, 0.0943, 0.4455	Best Loss: 0.3903 (4)	Accuracy: 87.68%
11	Validation losses: 0.3415, 0.0947, 0.4362	Best Loss: 0.3903 (5)	Accuracy: 87.72%
12	Validation losses: 0.3285, 0.0952, 0.4237	Best Loss: 0.3903 (6)	Accuracy: 88.36%
1

4	Validation losses: 0.3290, 0.0587, 0.3877	Best Loss: 0.3874 (1)	Accuracy: 88.42%
5	Validation losses: 0.3265, 0.0590, 0.3854	Best Loss: 0.3854 (0)	Accuracy: 88.44%
6	Validation losses: 0.2996, 0.0592, 0.3588	Best Loss: 0.3588 (0)	Accuracy: 88.94%
7	Validation losses: 0.3229, 0.0595, 0.3824	Best Loss: 0.3588 (1)	Accuracy: 88.30%
8	Validation losses: 0.3075, 0.0599, 0.3674	Best Loss: 0.3588 (2)	Accuracy: 89.06%
9	Validation losses: 0.3298, 0.0603, 0.3901	Best Loss: 0.3588 (3)	Accuracy: 89.26%
10	Validation losses: 0.3397, 0.0607, 0.4004	Best Loss: 0.3588 (4)	Accuracy: 88.36%
11	Validation losses: 0.3464, 0.0612, 0.4076	Best Loss: 0.3588 (5)	Accuracy: 87.68%
12	Validation losses: 0.3390, 0.0617, 0.4007	Best Loss: 0.3588 (6)	Accuracy: 88.30%
13	Validation losses: 0.3520, 0.0623, 0.4143	Best Loss: 0.3588 (7)	Accuracy: 88.76%
14	Validation losses: 0.3468, 0.0628, 0.4097	Best Loss: 0.3588 (8)	Accuracy: 87.60%
15	Validation losses: 0.3063, 0.0635, 0.3698	Best Loss: 0.3588 (9)	Accuracy: 89.42